# The Ridership Data

How is this ridership data calculated? Does it attempt to account for turnstile hoppers? How is it averaged? Are exceptional days (weekends, holidays) excepted or averaged in?

Answer: taken on non-summer weekdays (presumably different for each station)

There is next train info on ttc website.

Contact presto for better numbers? how to account for cash fares

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
# Station names
yus = ['sheppard west','wilson','yorkdale','lawrence west','glencairn','eglinton west','st clair west','dupont','spadina','st george','museum',"queen's park",'st patrick','osgoode','st andrew','union','king','queen','dundas','college','wellesley','bloor','rosedale','summerhill','st clair','davisville','eglinton','lawrence','york mills','sheppard','north york centre','finch']
bd = ['kipling','islington','royal york','old mill','jane','runnymede','high park','keele','dundas west','lansdowne','dufferin','ossington','christie','bathurst','spadina','st george','bay','yonge','sherbourne','castle frank','broadview','chester','pape','donlands','greenwood','coxwell','woodbine','main street','victoria park','warden','kennedy']
danger = ['st clair','lawrence','queen','dundas']
danger_corresp = ['st clair west','lawerence west',"queen's park",'dundas west']
interchange = ['spadina','st george','bloor','yonge'] # bloor-yonge is just one interchange, but it has different names on each line

In [89]:
# Information about time chunking
starts = np.array(['06:00','06:15','06:30','06:45','07:00','07:15','07:30','07:45','08:00',
          '08:15','08:30','08:45','09:00','09:15','09:30','09:45','10:00','10:30',
          '11:00','11:30','12:00','12:30','13:00','13:30','14:00','14:30','15:00',
          '15:15','15:30','15:45','16:00','16:15','16:30','16:45','17:00','17:15',
          '17:30','17:45','18:00','18:15','18:30','18:45','19:00','19:30','20:00',
          '20:30','21:00','21:30','22:00','22:30','23:00','23:30','24:00','24:30',
          '25:00'])

ends = np.array(['06:14','06:29','06:44','06:59','07:14','07:29','07:44','07:59','08:14',
        '08:29','08:44','08:59','09:14','09:29','09:44','09:59','10:14','10:29',
        '11:29','11:59','12:29','12:59','13:29','13:59','14:29','14:59','15:14',
        '15:29','15:44','15:59','16:14','16:29','16:44','16:59','17:14','17:29',
        '17:44','17:59','18:14','18:29','18:44','18:59','19:29','19:59','20:29',
        '20:59','21:29','21:59','22:29','22:59','23:29','23:59','24:29','24:59',
        '25:29'])

intervals = np.array([14]*(12+4))
intervals = np.append(intervals,np.array([29]*10))
intervals = np.append(intervals,np.array([14]*16))
intervals = np.append(intervals,np.array([29]*13))

trainint = {14:3,29:6}
trains = {14:np.floor(14./trainint[14]),29:np.floor(29./trainint[29])}

numbertrains = np.zeros(len(intervals))
traininterval = np.zeros(len(intervals))
for i,interval in enumerate(intervals):
    numbertrains[i] = trains[interval]
    traininterval[i] = trainint[interval]

In [94]:
intervals

array([14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 14, 14, 14, 14, 14, 14, 14, 14,
       14, 14, 14, 14, 14, 14, 14, 14, 29, 29, 29, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29])

In [93]:
numbertrains

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4.])

In [109]:
def read_station_data(stationlist,linenumber):
    stationinfo = pd.DataFrame()
    for station in stationlist:
        stationdata = np.loadtxt('data/{0}-{1}-2017.csv'.format(station,linenumber),delimiter=',',dtype='str')
        stationlabels = np.tile([station],len(starts))
        tostation = stationdata[:,1][:55]
        fromstation = stationdata[:,3][:55]
        totalstation = stationdata[:,5][:55]
        tempdf = pd.DataFrame()
        tempdf['station'] = stationlabels 
        tempdf['start'] = starts
        tempdf['end'] = ends
        tempdf['interval'] = intervals
        tempdf['to'] = tostation.astype(int)
        tempdf['from'] = fromstation.astype(int)
        tempdf['total'] = totalstation
        tempdf['dep_per_min'] = fromstation.astype(int)/intervals
        tempdf['arr_per_train'] = tostation.astype(int)/numbertrains
        tempdf['trains'] = traininterval
        stationinfo = stationinfo.append(tempdf,ignore_index=True)  
    return stationinfo

In [111]:
yusinfo = read_station_data(yus,1)

In [112]:
bdinfo = read_station_data(bd,2)

In [113]:
yusinfo

,station,start,end,interval,to,from,total,dep_per_min,arr_per_train,trains
0,sheppard west,06:00,06:14,14,170,1,171,0.071429,42.50,3.0
1,sheppard west,06:15,06:29,14,184,15,199,1.071429,46.00,3.0
2,sheppard west,06:30,06:44,14,235,200,435,14.285714,58.75,3.0
3,sheppard west,06:45,06:59,14,344,140,484,10.000000,86.00,3.0
4,sheppard west,07:00,07:14,14,502,183,685,13.071429,125.50,3.0
5,sheppard west,07:15,07:29,14,612,211,823,15.071429,153.00,3.0
6,sheppard west,07:30,07:44,14,613,360,973,25.714286,153.25,3.0
7,sheppard west,07:45,07:59,14,663,369,1032,26.357143,165.75,3.0
8,sheppard west,08:00,08:14,14,760,442,1202,31.571429,190.00,3.0
9,sheppard west,08:15,08:29,14,741,328,1069,23.428571,185.25,3.0


## Animation

Assume riders arrive at the value 'from' column divided by the time difference. Riders are picked up by trains that run at their scheduled intervals, and a number of riders equal to the value of the 'to' column divided by the number of trains in the interval. For now, assume trains every 3 minutes in the 14 minute increments and every 6 minutes otherwise.

### Yonge-University-Spadina

In [ ]:
yusinfo['']